### Exercise 3: 
A Fibonacci tree of height $h$ is an AVL tree with a minimum number of nodes among all AVL trees of height $h$.

(i) Familiarise yourself with the notion of Fibonacci tree by creating Fibonacci trees of height $1, 2, 3, 4$.

(ii) Write a program that checks, if a given binary tree with labelled vertices is a Fibonacci tree.

(iii) Determine the number of Fibonacci trees of height $h$ (up to isomorphism).

(iv) Give a criterion, whether it is possible to create a Fibonacci tree with $n$ given
elements.

(v) Implement a method that turns an AVL tree into a Fibonacci tree, if this is possible.

(vi) Implement a class FibTree of Fibonacci trees with a method insert, which according to (iv) must insert several elements. Exploit (v) for the definition of the insert method.

[link for your reference](https://ai.berkeley.edu/~cs61bl/r//cur/trees/fibonacci-tree.html?topic=lab15.topic&step=7&course)

In [1]:
# original AVL tree class change to Fibo
class FibTree:
    class FibNode:
        def __init__(self,item,balance=0,left=None,right=None):
            self.item = item
            self.balance = balance
            self.left = left
            self.right = right
            self.parent = None
        def getitem(self):
            return self.item
        def setitem(self,newitem):
            self.item = newitem
        def getbal(self):
            return self.balance
        def setbal(self,newbalance):
            self.balance = newbalance
        def getLeft(self):
            return self.left
        def setLeft(self,newleft):
            self.left = newleft
        def getRight(self):
            return self.right
        def setRight(self,newright):
            self.right = newright

        def __iter__(self):
            if self.left != None:
                for elem in self.left:
                    yield elem
            yield self.item, self.balance
            if self.right != None:
                for elem in self.right:
                    yield elem

        def __repr__(self):
            return "FibTree.FibNode("+repr(self.item)+",balance="+repr(self.balance)+",left="+repr(self.left)+",right="+repr(self.right)+")"

    def __init__(self):
        self.root = None

    def __iter__(self):
        if self.root != None:
            return self.root.__iter__()
        else:
            return [].__iter__()

    def __rotateLeft(node,child,nbal,cbal):
            node.setRight(child.getLeft())
            node.setbal(nbal)
            child.setLeft(node)
            child.setbal(cbal)
            return child
    def __rotateRight(node,child,nbal,cbal):
            node.setLeft(child.getRight())
            node.setbal(nbal)
            child.setRight(node)
            child.setbal(cbal)
            return child

    def insert(self,item):
        def __insert(root,item):
            newtree = FibTree.FibNode(item)
            stack = []
            badChild = None
            while root != None:
                if item < root.getitem():
                    stack.insert(0,(root,-1))
                    if root.getLeft() == None:
                        root.setLeft(newtree)
                        return (stack,badChild) # return the problem one
                    if root.getbal() == -1:
                        badChild = root.getLeft()
                    root = root.getLeft()
                else:       
                    stack.insert(0,(root,1))
                    if root.getRight() == None:
                        root.setRight(newtree)
                        return (stack,badChild) # return the problem one
                    if root.getbal() == 1:
                        badChild = root.getRight()
                    root = root.getRight()
    
        if self. root == None:
            self.root = FibTree.FibNode(item)
            stack = []
            badChild = None
        else:
            result = __insert(self.root,item)
            stack = result[0]
            badChild = result[1]
        rotTree = None
        for N in stack:
            node = N[0]
            inc = N[1]
            if rotTree != None:
                if inc == 1:
                    node.setRight(rotTree)
                else:
                    node.setLeft(rotTree)
                return
            
            newbal = node.getbal() + inc
            if badChild == node:
                if inc == 1:
                    badGrandchild = node.getRight()
                else:
                    badGrandchild = node.getLeft()
            if newbal == 0:
                node.setbal(0)
                return
            if -1 <= newbal <= 1:
                node.setbal(newbal)
            else:
                if inc == 1:
                    if badChild.getRight() == badGrandchild:
                        rotTree = FibTree.__rotateLeft(node,badChild,0,0)
                    else:
                        if item < badGrandchild.getitem():
                            n,c = 0,1
                        else:
                            if item > badGrandchild.getitem():
                                n,c = -1,0
                            else:
                                n,c = 0,0
                        rotTree1 = FibTree.__rotateRight(badChild,badGrandchild,c,0)
                        rotTree = FibTree.__rotateLeft(node,rotTree1,n,0)
                else:
                    if badChild.getLeft() == badGrandchild:
                        rotTree = FibTree.__rotateRight(node,badChild,0,0)
                    else:
                        if item < badGrandchild.getitem():
                            n,c = 0,-1
                        else:
                            if item > badGrandchild.getitem():
                                n,c = 1,0
                            else:
                                n,c = 0,0
                        rotTree1 = FibTree.__rotateLeft(badChild,badGrandchild,c,0)
                        rotTree = FibTree.__rotateRight(node,rotTree1,n,0)
        if rotTree != None:
            self.root = rotTree
    def find(self,item):
        def __find(root,item):
            if root == None:
                return False
            if item == root.getitem():
                return True
            if item < root.getitem():
                return __find(root.getLeft(),item)
            else:
                return __find(root.getRight(),item)
        return __find(self.root,item)
    
    def delete(self,item):
        def __findandreturn(root,item):
            if root == None:
                return([],root,False)
            if item == root.getitem():
                return([(root,0)],root,True)
            if item < root.getitem():
                result = __findandreturn(root.getLeft(),item)
                stack1 = result[0]
                stack1.append((root,1))
                return (stack1,result[1],result[2])
            else:
                result = __findandreturn(root.getRight(),item)
                stack1 = result[0]
                stack1.append((root,-1))
                return (stack1,result[1],result[2])

        def __findswapLeft(node,prev):
            stack = []
            while node.getLeft() != None:
                stack.insert(0,(node,1))
                prev = node
                node = node.getLeft()
            minitem = node.getitem()
            if stack == []:
                prev.setRight(node.getRight())
            else:
                prev.setLeft(node.getRight())
            return (minitem,stack)


        def __findswapRight(node,prev):
            stack = []
            while node.getRight() != None:
                stack.insert(0,(node,-1))
                prev = node
                node = node.getRight()
            maxitem = node.getitem()
            if stack == []:
                prev.setLeft(node.getRight())
            else:
                prev.setRight(node.getLeft())
            return (maxitem,stack)

        result = __findandreturn(self.root,item)
        if result[2] == False:
            return
        stack1 = result[0]
        start = stack1.pop(0)
        startnode = start[0]
        startinc = start[1]
        nextRight = startnode.getRight()
        nextLeft = startnode.getLeft()
        stack = []
        if startnode.getbal() == 1:
            result = __findswapLeft(nextRight,startnode)
            stack = result[1]
            startnode.setitem(result[0])
            startinc = -1
        else:
            if startnode.getbal() == -1:
                result = __findswapRight(nextLeft,startnode)
                stack = result[1]
                startnode.setitem(result[0])
                startinc = 1
            else:
                if startnode.getRight() != None:
                    if nextRight.getbal() != 1:
                        result = __findswapLeft(nextRight,startnode)
                        stack = result[1]
                        startnode.setitem(result[0])
                        startinc = -1
                    else:
                        result = __findswapRight(nextLeft,startnode)
                        stack = result[1]
                        startnode.setitem(result[0])
                        startinc = 1
                else:
                    if self.root == startnode:
                        self.root = None
                        return
                    last = stack1.pop(0)
                    lastnode = last[0]
                    if last[1] ==1:
                        lastnode.setLeft(None)
                    else:
                        lastnode.setRight(None)
                    stack1.insert(0,(lastnode,last[1]))
                    startnode = None
        if startnode != None:
            start = (startnode,startinc)
            stack1.insert(0,start)
        stack = stack + stack1
        stack2 = iter(stack+[(None,0)])
        NN = next(stack2)
        for N in stack:
            currentnode = N[0]
            increment = N[1]
            NN = next(stack2)
            rotTree = None
            stop = False
            newbal = currentnode.getbal() + increment
            if -1 <= newbal <= 1:
                currentnode.setbal(newbal)
                if newbal != 0:
                    return
            else:
                if increment == 1:
                    badChild = currentnode.getRight()
                    badGrandchild = badChild.getLeft()
                    if badChild.getbal() == 0:
                        rotTree = FibTree.__rotateLeft(currentnode,badChild,1,-1)
                        stop = True
                    else:
                        if badChild.getbal() == 1:
                            rotTree = FibTree.__rotateLeft(currentnode,badChild,0,0)
                        else:
                            nbal,cbal = 0,0
                            if badGrandchild.getbal() == 1:
                                nbal = -1
                            if badGrandchild.getbal() == -1:
                                cbal = 1
                            rotTree1 = FibTree.__rotateRight(badChild,badGrandchild,cbal,0)
                            rotTree = FibTree.__rotateLeft(currentnode,rotTree1,nbal,0)
                else:
                    badChild = currentnode.getLeft()
                    badGrandchild = badChild.getRight()
                    stop = False
                    if badChild.getbal() == 0:
                        rotTree = FibTree.__rotateRight(currentnode,badChild,-1,1)
                        stop = True
                    else:
                        if badChild.getbal() == -1:
                            rotTree = FibTree.__rotateRight(currentnode,badChild,0,0)
                        else:
                            nbal,cbal = 0,0
                            if badGrandchild.getbal() == 1:
                                cbal = -1
                            if badGrandchild.getbal() == -1:
                                nbal = 1
                            rotTree1 = FibTree.__rotateLeft(badChild,badGrandchild,cbal,0)
                            rotTree = FibTree.__rotateRight(currentnode,rotTree1,nbal,0)
            if rotTree != None:
                if NN[0] != None:
                    if NN[1] == 1:
                        NN[0].setLeft(rotTree)
                    if NN[1] == -1:
                        NN[0].setRight(rotTree)
                else:
                    self.root = rotTree
            if stop == True:
                return
    
    ## Add a function to print out the tree intuitionistically
    def print_Fib(self):
        ret = self.print_tree(self.root, 0, 0,{})
        ret = sorted(ret.items(), key=lambda x: x[1])
        cur_x = 0
        cur_y = 0
        for item in ret:
            x = item[1][1]
            y = item[1][0]
            parent = item[1][2]
            for i in range(cur_y, y):
                print('\n')
                cur_x = 0
            for i in range(cur_x, x):
                print(' ', end='')
            print(item[0], end='')
            print('('+str(parent)+')',end = '')
            cur_x = x+1
            cur_y = y
        print('\n')
    # helper function
    def print_tree(self, root, x, y, ret):
        # none do nothing and return
        if(root == None):
            return ret
        if(root.left == None and root.right == None): # leaf node   
            ret[str(root.item)] = (y, x, root.parent.item)
            return ret
        # otherwise, the left or right node is not None
        if(x-8 < 0):
            x = 8
        if(root.left != None):
            root.left.parent = root
            ret = self.print_tree(root.left, x-8, y+1, ret)
            leftChild = str(root.left.item)
            if(root == self.root):
                x = ret[leftChild][1]+16 # to distinct left and right tree
            else:
                x = ret[leftChild][1]+8
        if(root.parent == None):
            ret[str(root.item)] = (y, x, None)
        else:
            ret[str(root.item)] = (y, x, root.parent.item)
        if(root.right != None):
            root.right.parent = root
            if(root == self.root):
                ret = self.print_tree(root.right, x+16, y+1, ret) # to distinct left and right tree
            else:
                ret = self.print_tree(root.right, x+8, y+1, ret)
        return ret
    
    def height(self, node):
        if(node == None):
            return -1
        left_balance = self.height(node.left)
        right_balance = self.height(node.right)
        return max(left_balance, right_balance)+1
        
    

In [3]:
# (1)
print('---------h = 2, insert (1,2) , 2---------')
Fib_h1 = FibTree()
Fib_h1.insert(1)
Fib_h1.insert(2)
Fib_h1.print_Fib()
print('---------h = 3, insert (0,1,2,4), 4---------')
Fib_h2 = FibTree()
Fib_h2.insert(1)
Fib_h2.insert(2)
Fib_h2.insert(4)
Fib_h2.insert(0)
Fib_h2.print_Fib()
print('---------h = 4, insert (1,0,2,-1,3,4,5), 7---------')
Fib_h3 = FibTree()
Fib_h3.insert(1)
Fib_h3.insert(0)
Fib_h3.insert(2)
Fib_h3.insert(-1)
Fib_h3.insert(3)
Fib_h3.insert(4)
Fib_h3.insert(5)
Fib_h3.print_Fib()
print('---------h = 5, insert (1,0,2,-1,-0.5,3,4,-2,3.5,4.5,5,3.25), 12---------')
Fib_h4 = FibTree()
Fib_h4.insert(1)
Fib_h4.insert(0)
Fib_h4.insert(2)
Fib_h4.insert(-1)
Fib_h4.insert(-0.5)
Fib_h4.insert(3)
Fib_h4.insert(4)
Fib_h4.insert(-2)
Fib_h4.insert(3.5)
Fib_h4.insert(4.5)
Fib_h4.insert(5)
Fib_h4.insert(3.25)
Fib_h4.print_Fib()

---------h = 2, insert (1,2) , 2---------
        1(None)

                        2(1)

---------h = 3, insert (0,1,2,4), 4---------
                        2(None)

        1(2)                               4(2)

0(1)

---------h = 4, insert (1,0,2,-1,3,4,5), 7---------
                        1(None)

        0(1)                               3(1)

-1(0)                               2(3)               4(3)

                                                        5(4)

---------h = 5, insert (1,0,2,-1,-0.5,3,4,-2,3.5,4.5,5,3.25), 12---------
                                1(None)

                -0.5(1)                               4(1)

        -1(-0.5)               0(-0.5)               3(4)               4.5(4)

-2(-1)                               2(3)               3.5(3)               5(4.5)

                                        3.25(3.5)



#### Recursion Relation: $L(n) = L(n-1) + L(n-2) + 1$

So for $h=4$, $L(4) = 1 + 2 + 4 = 7$

Base cases, $L(1) = 1, L(0) = 0, L(2) = 1+0+1 = 2$


In [9]:
# (2)
# Definition of BST
class BST:
    class __Node:
        def __init__(self,val,left=None,right=None):
            self.val = val
            self.left = left
            self.right = right
            self.parent = None
        def getVal(self):
            return self.val
        def setVal(self,newval):
            self.val = newval
        def getLeft(self):
            return self.left
        def setLeft(self,newleft):
            self.left = newleft
        def getRight(self):
            return self.right
        def setRight(self,newright):
            self.right = newright 
        def __iter__(self):
            if self.left != None:
                for elem in self.left:
                    yield elem
            yield self.val
            if self.right != None:
                for elem in self.right:
                    yield elem
                    
    def __init__(self):
        self.root = None
    def insert(self,val):
        def __insert(root,val):
            if root == None:
                return BST.__Node(val)
            if val < root.getVal():
                root.setLeft(__insert(root.getLeft(),val))
            else:
                root.setRight(__insert(root.getRight(),val))
            return root
        self.root = __insert(self.root,val)
    def __iter__(self):
        if self.root != None:
            return self.root.__iter__()
        else:
            return [].__iter__()
    def find(self,val):
        def __find(root,val):
            if root == None:
                return False
            if val == root.getVal():
                return True
            if val < root.getVal():
                return __find(root.getLeft(),val)
            else:
                return __find(root.getRight(),val)
        return __find(self.root,val)
    def delete(self,val):
        def __merge(leftnode,rightnode):
            if rightnode == None:
                return leftnode
            if rightnode.getLeft() == None:
                rightnode.setLeft(leftnode)
                return rightnode
            __merge(leftnode,rightnode.getLeft())
            return rightnode
        def __delete(root,val):
            if root == None:
                return root
            if val == root.getVal():
                return __merge(root.getLeft(),root.getRight())
            if val < root.getVal():
                root.setLeft(__delete(root.getLeft(),val))
                return root
            root.setRight(__delete(root.getRight(),val))
            return root
        __delete(self.root,val)
    ## Add a function to print out the tree intuitionistically
    def print_BST(self):
        ret = self.print_tree(self.root, 0, 0,{})
        ret = sorted(ret.items(), key=lambda x: x[1])
        cur_x = 0
        cur_y = 0
        for item in ret:
            x = item[1][1]
            y = item[1][0]
            parent = item[1][2]
            for i in range(cur_y, y):
                print('\n')
                cur_x = 0
            for i in range(cur_x, x):
                print(' ', end='')
            print(item[0], end='')
            print('('+str(parent)+')',end = '')
            cur_x = x+1
            cur_y = y
        print('\n')
        
    # helper function
    def print_tree(self, root, x, y, ret):
        # none do nothing and return
        if(root == None):
            return ret
        if(root.left == None and root.right == None): # leaf node   
            ret[str(root.val)] = (y, x, root.parent.val)
            return ret
        # otherwise, the left or right node is not None
        if(x-8 < 0):
            x = 8
        if(root.left != None):
            root.left.parent = root
            ret = self.print_tree(root.left, x-8, y+1, ret)
            leftChild = str(root.left.val)
            if(root == self.root):
                x = ret[leftChild][1]+16 # to distinct left and right tree
            else:
                x = ret[leftChild][1]+8
        if(root.parent == None):
            ret[str(root.val)] = (y, x, None)
        else:
            ret[str(root.val)] = (y, x, root.parent.val)
        if(root.right != None):
            root.right.parent = root
            if(root == self.root):
                ret = self.print_tree(root.right, x+16, y+1, ret) # to distinct left and right tree
            else:
                ret = self.print_tree(root.right, x+8, y+1, ret)
        return ret
    # get the hight of a specified node recursively
    def height(self, node):
        if(node == None):
            return -1
        left_balance = self.height(node.left)
        right_balance = self.height(node.right)
        return max(left_balance, right_balance)+1
    
    # return whether a tree is AVL tree
    def isAVL(self, root):
        if(root == None):
            return True
        R_height = self.height(root.right)
        L_height = self.height(root.left)
        if(abs(R_height-L_height)>1):
            print('for node', root.val, 'Right height:', R_height, 'Left height:', L_height, 'not almost balanced!')
            return False
        R_AVL = self.isAVL(root.right)
        L_AVL = self.isAVL(root.left)
        if(R_AVL== False or L_AVL==False):
            return False
        return True
    
    # Fibonnaci Number 
    def __Fibo(height):
        if(height==0):
            return 1
        if(height==1):
            return 2
        return 1+ BST.__Fibo(height-1)+ BST.__Fibo(height-2)
    
    # Node Number of a given tree
    def __NodeNumber(root):
        if(root == None):
            return 0
        left = BST.__NodeNumber(root.left)
        right = BST.__NodeNumber(root.right)
        return left+right+1
    
    def isFiboTree(self):
        if(not self.isAVL(self.root)):
            return False
        fibo = BST.__Fibo(self.height(self.root))
        print('True number of Nodes:', fibo)
        node_num = BST.__NodeNumber(self.root)
        print('Real number of Nodes:', node_num)
        if(node_num!=fibo):
            print('not equal, not a fibonacci tree')
            return False
        return True  
        

In [10]:
# try for the not true case
print('--------- insert (1,0,2,-1,3,4,5), 7---------')
B = BST()
B.insert(4)
B.insert(3)
B.insert(5)
B.insert(2)
B.insert(3.5)
B.insert(4.5)
B.insert(6)
B.print_BST()
B.isFiboTree()
print('--------- insert (1,2,3,4,5,6,7,8,9,10), 7---------')
B = BST()
for i in range(10):
    B.insert(i)
B.print_BST()
B.isFiboTree()

--------- insert (1,0,2,-1,3,4,5), 7---------
                        4(None)

        3(4)                               5(4)

2(3)               3.5(3)               4.5(5)               6(5)

True number of Nodes: 4
Real number of Nodes: 7
not equal, not a fibonacci tree
--------- insert (1,2,3,4,5,6,7,8,9,10), 7---------
        0(None)

                        1(0)

                                2(1)

                                        3(2)

                                                4(3)

                                                        5(4)

                                                                6(5)

                                                                        7(6)

                                                                                8(7)

                                                                                        9(8)

for node 0 Right height: 8 Left height: -1 not almost balanced!


False

In [11]:
# try for the true case
print('--------- insert (1,2), 2---------')
B = BST()
B.insert(1)
B.insert(2)
B.print_BST()
B.isFiboTree()
print('--------- insert (2,1,4,0), 4---------')
B = BST()
B.insert(2)
B.insert(1)
B.insert(4)
B.insert(0)
B.print_BST()
B.isFiboTree()

--------- insert (1,2), 2---------
        1(None)

                        2(1)

True number of Nodes: 2
Real number of Nodes: 2
--------- insert (2,1,4,0), 4---------
                        2(None)

        1(2)                               4(2)

0(1)

True number of Nodes: 4
Real number of Nodes: 4


True

#### (3) Determine the number of Fibonacci trees of height $h$ (up to isomorphism).
$h = 1, N(1) = 2$

$h = 2, N(2) = 4$

$h = 3, N(3) = 16$

Thus, when $H = h, N(h) = 2^\mathrm{(number\ of\ non-leaf\ node)}$.

For each non-leaf node, we can flip the left and right subtree of it without violating the AVL tree definition. For each height, the leaf node number is a fibonacci number with $F(n) = F(n-1) + F(n-2)$.

Total number of Nodes: $L(n) = L(n-1) + L(n-2) + 1$

More accurately, $N(h) = 2^{L(n) - F(n)}$

Or we can present $N(n-1), N(n-2)$ as the isomorphisms of FiboTree of height $h-1, h-2$ resprectively and they are left and right subtrees of a tree with height h. So for this root, we can filp the right and left subtree without violating the AVL tree definition. 

Thus, the recursion is simply $N(n) = 2 * N(n-1) * N(n-2)$.

#### (iv) Give a criterion, whether it is possible to create a Fibonacci tree with $n$ given elements.
We only need to check if $n$ equals to some $L(x)$ (number of nodes for Fibonacci tree of height $x$).

#### (v) Implement a method that turns an AVL tree into a Fibonacci tree, if this is possible.
Firstly, $n$ should equal to some $L(x)$ (number of nodes for Fibonacci tree of height $x$), where $x$ is the height of the given tree. Besides, we should also check if the given tree also has a height of $x$.

If we consider the more detailed structure of the tree, we should make sure one of subtree of the root has height $(x-1)$ and the other $(x-2)$.

Fibonacci trees have several properties that can be used to form a compact Fibonacci tree:

1.Every node in a Fibonacci tree is itself a Fibonacci tree.

2.The number of nodes in a Fibonacci tree of height $n$ is equal to $F_{n+3} - 1$.

3.The number of nodes in between a node and its left child is equal to the number of nodes in the node's left child's right child. 

4.The number of nodes in between a node and its right child is equal to the number of nodes in the node's right child's left child.

In [4]:
class AVLTree:
    class AVLNode:
        def __init__(self,item,balance=0,left=None,right=None):
            self.item = item
            self.balance = balance
            self.left = left
            self.right = right
        def getitem(self):
            return self.item
        def setitem(self,newitem):
            self.item = newitem
        def getbal(self):
            return self.balance
        def setbal(self,newbalance):
            self.balance = newbalance
        def getLeft(self):
            return self.left
        def setLeft(self,newleft):
            self.left = newleft
        def getRight(self):
            return self.right
        def setRight(self,newright):
            self.right = newright

        def __iter__(self):
            if self.left != None:
                for elem in self.left:
                    yield elem
            yield self.item, self.balance
            if self.right != None:
                for elem in self.right:
                    yield elem

        def __repr__(self):
            return "AVLTree.AVLNode("+repr(self.item)+",balance="+repr(self.balance)+",left="+repr(self.left)+",right="+repr(self.right)+")"

    def __init__(self):
        self.root = None

    def __iter__(self):
        if self.root != None:
            return self.root.__iter__()
        else:
            return [].__iter__()

    def __rotateLeft(node,child,nbal,cbal):
            node.setRight(child.getLeft())
            node.setbal(nbal)
            child.setLeft(node)
            child.setbal(cbal)
            return child
    def __rotateRight(node,child,nbal,cbal):
            node.setLeft(child.getRight())
            node.setbal(nbal)
            child.setRight(node)
            child.setbal(cbal)
            return child

    def insert(self,item):
        def __insert(root,item):
            newtree = AVLTree.AVLNode(item)
            stack = []
            badChild = None
            while root != None:
                if item < root.getitem():
                    stack.insert(0,(root,-1))
                    if root.getLeft() == None:
                        root.setLeft(newtree)
                        return (stack,badChild) # return the problem one
                    if root.getbal() == -1:
                        badChild = root.getLeft()
                    root = root.getLeft()
                else:       
                    stack.insert(0,(root,1))
                    if root.getRight() == None:
                        root.setRight(newtree)
                        return (stack,badChild) # return the problem one
                    if root.getbal() == 1:
                        badChild = root.getRight()
                    root = root.getRight()
    
        if self. root == None:
            self.root = AVLTree.AVLNode(item)
            stack = []
            badChild = None
        else:
            result = __insert(self.root,item)
            stack = result[0]
            badChild = result[1]
        rotTree = None
        for N in stack:
            node = N[0]
            inc = N[1]
            if rotTree != None:
                if inc == 1:
                    node.setRight(rotTree)
                else:
                    node.setLeft(rotTree)
                return
            
            newbal = node.getbal() + inc
            if badChild == node:
                if inc == 1:
                    badGrandchild = node.getRight()
                else:
                    badGrandchild = node.getLeft()
            if newbal == 0:
                node.setbal(0)
                return
            if -1 <= newbal <= 1:
                node.setbal(newbal)
            else:
                if inc == 1:
                    if badChild.getRight() == badGrandchild:
                        rotTree = AVLTree.__rotateLeft(node,badChild,0,0)
                    else:
                        if item < badGrandchild.getitem():
                            n,c = 0,1
                        else:
                            if item > badGrandchild.getitem():
                                n,c = -1,0
                            else:
                                n,c = 0,0
                        rotTree1 = AVLTree.__rotateRight(badChild,badGrandchild,c,0)
                        rotTree = AVLTree.__rotateLeft(node,rotTree1,n,0)
                else:
                    if badChild.getLeft() == badGrandchild:
                        rotTree = AVLTree.__rotateRight(node,badChild,0,0)
                    else:
                        if item < badGrandchild.getitem():
                            n,c = 0,-1
                        else:
                            if item > badGrandchild.getitem():
                                n,c = 1,0
                            else:
                                n,c = 0,0
                        rotTree1 = AVLTree.__rotateLeft(badChild,badGrandchild,c,0)
                        rotTree = AVLTree.__rotateRight(node,rotTree1,n,0)
        if rotTree != None:
            self.root = rotTree
    def find(self,item):
        def __find(root,item):
            if root == None:
                return False
            if item == root.getitem():
                return True
            if item < root.getitem():
                return __find(root.getLeft(),item)
            else:
                return __find(root.getRight(),item)
        return __find(self.root,item)
    
    def delete(self,item):
        def __findandreturn(root,item):
            if root == None:
                return([],root,False)
            if item == root.getitem():
                return([(root,0)],root,True)
            if item < root.getitem():
                result = __findandreturn(root.getLeft(),item)
                stack1 = result[0]
                stack1.append((root,1))
                return (stack1,result[1],result[2])
            else:
                result = __findandreturn(root.getRight(),item)
                stack1 = result[0]
                stack1.append((root,-1))
                return (stack1,result[1],result[2])

        def __findswapLeft(node,prev):
            stack = []
            while node.getLeft() != None:
                stack.insert(0,(node,1))
                prev = node
                node = node.getLeft()
            minitem = node.getitem()
            if stack == []:
                prev.setRight(node.getRight())
            else:
                prev.setLeft(node.getRight())
            return (minitem,stack)


        def __findswapRight(node,prev):
            stack = []
            while node.getRight() != None:
                stack.insert(0,(node,-1))
                prev = node
                node = node.getRight()
            maxitem = node.getitem()
            if stack == []:
                prev.setLeft(node.getRight())
            else:
                prev.setRight(node.getLeft())
            return (maxitem,stack)

        result = __findandreturn(self.root,item)
        if result[2] == False:
            return
        stack1 = result[0]
        start = stack1.pop(0)
        startnode = start[0]
        startinc = start[1]
        nextRight = startnode.getRight()
        nextLeft = startnode.getLeft()
        stack = []
        if startnode.getbal() == 1:
            result = __findswapLeft(nextRight,startnode)
            stack = result[1]
            startnode.setitem(result[0])
            startinc = -1
        else:
            if startnode.getbal() == -1:
                result = __findswapRight(nextLeft,startnode)
                stack = result[1]
                startnode.setitem(result[0])
                startinc = 1
            else:
                if startnode.getRight() != None:
                    if nextRight.getbal() != 1:
                        result = __findswapLeft(nextRight,startnode)
                        stack = result[1]
                        startnode.setitem(result[0])
                        startinc = -1
                    else:
                        result = __findswapRight(nextLeft,startnode)
                        stack = result[1]
                        startnode.setitem(result[0])
                        startinc = 1
                else:
                    if self.root == startnode:
                        self.root = None
                        return
                    last = stack1.pop(0)
                    lastnode = last[0]
                    if last[1] ==1:
                        lastnode.setLeft(None)
                    else:
                        lastnode.setRight(None)
                    stack1.insert(0,(lastnode,last[1]))
                    startnode = None
        if startnode != None:
            start = (startnode,startinc)
            stack1.insert(0,start)
        stack = stack + stack1
        stack2 = iter(stack+[(None,0)])
        NN = next(stack2)
        for N in stack:
            currentnode = N[0]
            increment = N[1]
            NN = next(stack2)
            rotTree = None
            stop = False
            newbal = currentnode.getbal() + increment
            if -1 <= newbal <= 1:
                currentnode.setbal(newbal)
                if newbal != 0:
                    return
            else:
                if increment == 1:
                    badChild = currentnode.getRight()
                    badGrandchild = badChild.getLeft()
                    if badChild.getbal() == 0:
                        rotTree = FibTree.__rotateLeft(currentnode,badChild,1,-1)
                        stop = True
                    else:
                        if badChild.getbal() == 1:
                            rotTree = FibTree.__rotateLeft(currentnode,badChild,0,0)
                        else:
                            nbal,cbal = 0,0
                            if badGrandchild.getbal() == 1:
                                nbal = -1
                            if badGrandchild.getbal() == -1:
                                cbal = 1
                            rotTree1 = FibTree.__rotateRight(badChild,badGrandchild,cbal,0)
                            rotTree = FibTree.__rotateLeft(currentnode,rotTree1,nbal,0)
                else:
                    badChild = currentnode.getLeft()
                    badGrandchild = badChild.getRight()
                    stop = False
                    if badChild.getbal() == 0:
                        rotTree = FibTree.__rotateRight(currentnode,badChild,-1,1)
                        stop = True
                    else:
                        if badChild.getbal() == -1:
                            rotTree = FibTree.__rotateRight(currentnode,badChild,0,0)
                        else:
                            nbal,cbal = 0,0
                            if badGrandchild.getbal() == 1:
                                cbal = -1
                            if badGrandchild.getbal() == -1:
                                nbal = 1
                            rotTree1 = FibTree.__rotateLeft(badChild,badGrandchild,cbal,0)
                            rotTree = FibTree.__rotateRight(currentnode,rotTree1,nbal,0)
            if rotTree != None:
                if NN[0] != None:
                    if NN[1] == 1:
                        NN[0].setLeft(rotTree)
                    if NN[1] == -1:
                        NN[0].setRight(rotTree)
                else:
                    self.root = rotTree
            if stop == True:
                return
    # get the hight of a specified node recursively
    def height(self, node):
        if(node == None):
            return -1
        left_balance = self.height(node.left)
        right_balance = self.height(node.right)
        return max(left_balance, right_balance)+1
    
    ## add a function AVLToFibTree()
    def AVLToFibTree(self):
    
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-4-566510b91b8b>, line 317)

*Note for (v)*:  
The whole point is to keep **the number of elements** in the left and right subtree of each node as $L(h-1)$ and $L(h-2)$. We do the recursion starting from the root and go downward.  
We don't need to worry about the internal structure of the subtrees, as the recursion will always end at a subtree with $4$ or $2$ elements, which has a fixed shape.

#### (vi) Implement a class FibTree of Fibonacci trees with a method insert, which according to (iv) must insert several elements. Exploit (v) for the definition of the insert method.

[link for reference](https://cloud.tencent.com/developer/ask/43539)

In [ ]:
# original AVL tree class
class FibTree:
    class FibNode:
        def __init__(self,item,balance=0,left=None,right=None):
            self.item = item
            self.balance = balance
            self.left = left
            self.right = right
            self.parent = None
        def getitem(self):
            return self.item
        def setitem(self,newitem):
            self.item = newitem
        def getbal(self):
            return self.balance
        def setbal(self,newbalance):
            self.balance = newbalance
        def getLeft(self):
            return self.left
        def setLeft(self,newleft):
            self.left = newleft
        def getRight(self):
            return self.right
        def setRight(self,newright):
            self.right = newright

        def __iter__(self):
            if self.left != None:
                for elem in self.left:
                    yield elem
            yield self.item, self.balance
            if self.right != None:
                for elem in self.right:
                    yield elem

        def __repr__(self):
            return "FibTree.FibNode("+repr(self.item)+",balance="+repr(self.balance)+",left="+repr(self.left)+",right="+repr(self.right)+")"

    def __init__(self):
        self.root = None

    def __iter__(self):
        if self.root != None:
            return self.root.__iter__()
        else:
            return [].__iter__()

    def __rotateLeft(node,child,nbal,cbal):
            node.setRight(child.getLeft())
            node.setbal(nbal)
            child.setLeft(node)
            child.setbal(cbal)
            return child
    def __rotateRight(node,child,nbal,cbal):
            node.setLeft(child.getRight())
            node.setbal(nbal)
            child.setRight(node)
            child.setbal(cbal)
            return child

    def insert(self,item):
        def __insert(root,item):
            newtree = FibTree.FibNode(item)
            stack = []
            badChild = None
            while root != None:
                if item < root.getitem():
                    stack.insert(0,(root,-1))
                    if root.getLeft() == None:
                        root.setLeft(newtree)
                        return (stack,badChild) # return the problem one
                    if root.getbal() == -1:
                        badChild = root.getLeft()
                    root = root.getLeft()
                else:       
                    stack.insert(0,(root,1))
                    if root.getRight() == None:
                        root.setRight(newtree)
                        return (stack,badChild) # return the problem one
                    if root.getbal() == 1:
                        badChild = root.getRight()
                    root = root.getRight()
    
        if self. root == None:
            self.root = FibTree.FibNode(item)
            stack = []
            badChild = None
        else:
            result = __insert(self.root,item)
            stack = result[0]
            badChild = result[1]
        rotTree = None
        for N in stack:
            node = N[0]
            inc = N[1]
            if rotTree != None:
                if inc == 1:
                    node.setRight(rotTree)
                else:
                    node.setLeft(rotTree)
                return
            
            newbal = node.getbal() + inc
            if badChild == node:
                if inc == 1:
                    badGrandchild = node.getRight()
                else:
                    badGrandchild = node.getLeft()
            if newbal == 0:
                node.setbal(0)
                return
            if -1 <= newbal <= 1:
                node.setbal(newbal)
            else:
                if inc == 1:
                    if badChild.getRight() == badGrandchild:
                        rotTree = FibTree.__rotateLeft(node,badChild,0,0)
                    else:
                        if item < badGrandchild.getitem():
                            n,c = 0,1
                        else:
                            if item > badGrandchild.getitem():
                                n,c = -1,0
                            else:
                                n,c = 0,0
                        rotTree1 = FibTree.__rotateRight(badChild,badGrandchild,c,0)
                        rotTree = FibTree.__rotateLeft(node,rotTree1,n,0)
                else:
                    if badChild.getLeft() == badGrandchild:
                        rotTree = FibTree.__rotateRight(node,badChild,0,0)
                    else:
                        if item < badGrandchild.getitem():
                            n,c = 0,-1
                        else:
                            if item > badGrandchild.getitem():
                                n,c = 1,0
                            else:
                                n,c = 0,0
                        rotTree1 = FibTree.__rotateLeft(badChild,badGrandchild,c,0)
                        rotTree = FibTree.__rotateRight(node,rotTree1,n,0)
        if rotTree != None:
            self.root = rotTree
    def find(self,item):
        def __find(root,item):
            if root == None:
                return False
            if item == root.getitem():
                return True
            if item < root.getitem():
                return __find(root.getLeft(),item)
            else:
                return __find(root.getRight(),item)
        return __find(self.root,item)
    
    def delete(self,item):
        def __findandreturn(root,item):
            if root == None:
                return([],root,False)
            if item == root.getitem():
                return([(root,0)],root,True)
            if item < root.getitem():
                result = __findandreturn(root.getLeft(),item)
                stack1 = result[0]
                stack1.append((root,1))
                return (stack1,result[1],result[2])
            else:
                result = __findandreturn(root.getRight(),item)
                stack1 = result[0]
                stack1.append((root,-1))
                return (stack1,result[1],result[2])

        def __findswapLeft(node,prev):
            stack = []
            while node.getLeft() != None:
                stack.insert(0,(node,1))
                prev = node
                node = node.getLeft()
            minitem = node.getitem()
            if stack == []:
                prev.setRight(node.getRight())
            else:
                prev.setLeft(node.getRight())
            return (minitem,stack)


        def __findswapRight(node,prev):
            stack = []
            while node.getRight() != None:
                stack.insert(0,(node,-1))
                prev = node
                node = node.getRight()
            maxitem = node.getitem()
            if stack == []:
                prev.setLeft(node.getRight())
            else:
                prev.setRight(node.getLeft())
            return (maxitem,stack)

        result = __findandreturn(self.root,item)
        if result[2] == False:
            return
        stack1 = result[0]
        start = stack1.pop(0)
        startnode = start[0]
        startinc = start[1]
        nextRight = startnode.getRight()
        nextLeft = startnode.getLeft()
        stack = []
        if startnode.getbal() == 1:
            result = __findswapLeft(nextRight,startnode)
            stack = result[1]
            startnode.setitem(result[0])
            startinc = -1
        else:
            if startnode.getbal() == -1:
                result = __findswapRight(nextLeft,startnode)
                stack = result[1]
                startnode.setitem(result[0])
                startinc = 1
            else:
                if startnode.getRight() != None:
                    if nextRight.getbal() != 1:
                        result = __findswapLeft(nextRight,startnode)
                        stack = result[1]
                        startnode.setitem(result[0])
                        startinc = -1
                    else:
                        result = __findswapRight(nextLeft,startnode)
                        stack = result[1]
                        startnode.setitem(result[0])
                        startinc = 1
                else:
                    if self.root == startnode:
                        self.root = None
                        return
                    last = stack1.pop(0)
                    lastnode = last[0]
                    if last[1] ==1:
                        lastnode.setLeft(None)
                    else:
                        lastnode.setRight(None)
                    stack1.insert(0,(lastnode,last[1]))
                    startnode = None
        if startnode != None:
            start = (startnode,startinc)
            stack1.insert(0,start)
        stack = stack + stack1
        stack2 = iter(stack+[(None,0)])
        NN = next(stack2)
        for N in stack:
            currentnode = N[0]
            increment = N[1]
            NN = next(stack2)
            rotTree = None
            stop = False
            newbal = currentnode.getbal() + increment
            if -1 <= newbal <= 1:
                currentnode.setbal(newbal)
                if newbal != 0:
                    return
            else:
                if increment == 1:
                    badChild = currentnode.getRight()
                    badGrandchild = badChild.getLeft()
                    if badChild.getbal() == 0:
                        rotTree = FibTree.__rotateLeft(currentnode,badChild,1,-1)
                        stop = True
                    else:
                        if badChild.getbal() == 1:
                            rotTree = FibTree.__rotateLeft(currentnode,badChild,0,0)
                        else:
                            nbal,cbal = 0,0
                            if badGrandchild.getbal() == 1:
                                nbal = -1
                            if badGrandchild.getbal() == -1:
                                cbal = 1
                            rotTree1 = FibTree.__rotateRight(badChild,badGrandchild,cbal,0)
                            rotTree = FibTree.__rotateLeft(currentnode,rotTree1,nbal,0)
                else:
                    badChild = currentnode.getLeft()
                    badGrandchild = badChild.getRight()
                    stop = False
                    if badChild.getbal() == 0:
                        rotTree = FibTree.__rotateRight(currentnode,badChild,-1,1)
                        stop = True
                    else:
                        if badChild.getbal() == -1:
                            rotTree = FibTree.__rotateRight(currentnode,badChild,0,0)
                        else:
                            nbal,cbal = 0,0
                            if badGrandchild.getbal() == 1:
                                cbal = -1
                            if badGrandchild.getbal() == -1:
                                nbal = 1
                            rotTree1 = FibTree.__rotateLeft(badChild,badGrandchild,cbal,0)
                            rotTree = FibTree.__rotateRight(currentnode,rotTree1,nbal,0)
            if rotTree != None:
                if NN[0] != None:
                    if NN[1] == 1:
                        NN[0].setLeft(rotTree)
                    if NN[1] == -1:
                        NN[0].setRight(rotTree)
                else:
                    self.root = rotTree
            if stop == True:
                return
    
    ## Add a function to print out the tree intuitionistically
    def print_Fib(self):
        ret = self.print_tree(self.root, 0, 0,{})
        ret = sorted(ret.items(), key=lambda x: x[1])
        cur_x = 0
        cur_y = 0
        for item in ret:
            x = item[1][1]
            y = item[1][0]
            parent = item[1][2]
            for i in range(cur_y, y):
                print('\n')
                cur_x = 0
            for i in range(cur_x, x):
                print(' ', end='')
            print(item[0], end='')
            print('('+str(parent)+')',end = '')
            cur_x = x+1
            cur_y = y
        print('\n')
    # helper function
    def print_tree(self, root, x, y, ret):
        # none do nothing and return
        if(root == None):
            return ret
        if(root.left == None and root.right == None): # leaf node   
            ret[str(root.item)] = (y, x, root.parent.item)
            return ret
        # otherwise, the left or right node is not None
        if(x-8 < 0):
            x = 8
        if(root.left != None):
            root.left.parent = root
            ret = self.print_tree(root.left, x-8, y+1, ret)
            leftChild = str(root.left.item)
            if(root == self.root):
                x = ret[leftChild][1]+16 # to distinct left and right tree
            else:
                x = ret[leftChild][1]+8
        if(root.parent == None):
            ret[str(root.item)] = (y, x, None)
        else:
            ret[str(root.item)] = (y, x, root.parent.item)
        if(root.right != None):
            root.right.parent = root
            if(root == self.root):
                ret = self.print_tree(root.right, x+16, y+1, ret) # to distinct left and right tree
            else:
                ret = self.print_tree(root.right, x+8, y+1, ret)
        return ret
    
    def height(self, node):
        if(node == None):
            return -1
        left_balance = self.height(node.left)
        right_balance = self.height(node.right)
        return max(left_balance, right_balance)+1
        
    

*Note for (vi)*:  
An intuitive idea is shown below:
First, keep **the number of element that should be inserted** as a class property, then we can use it to check if the length of inserted sequence meets the criterion.  
If the length of inserted sequnece is correct, then we can just insert the elements like in a normal AVL tree. After that, we can use the method in (v) to transform the resulitng AVL tree into a Fibonacci tree.